In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

# External Datasets

Load in datasets for population, urbanization and gdp values for the US and Canada.

In [2]:
population = pd.read_csv('population.csv')
urbanization = pd.read_csv('urbanization.csv',index_col=0)
gdp = pd.read_csv('gdp.csv')

# Internal Datasets

Load in the mapper dataset, and our data.

In [3]:
mapper = pd.read_csv('Dataset_2.0_09222021_MF_UF.csv')
building_name_mapper = pd.read_excel('BuildingTypeNames.xlsx')

In [4]:
df = pd.read_excel('../Dataset/dataset.xlsx',header=3,index_col=1)

In [5]:
df = df.drop('Unnamed: 0',axis=1).T.reset_index().rename({'Building Identifier':'index','index':'Building Identifier'},axis=1)

In [6]:
df = df[~df['Building Identifier'].str.contains('\.')]
df = df[df['Building Identifier'].str.contains('0')]

# Additional Categories

Specify the mapping from additional category codes to their categories, and also list the columns which are not materials.

In [7]:
additional_categories_map = {v:k for k,v in {
    'Continuous Footings':'0CF',
    'Foundation Walls':'0FW',
    'Spread Footings':'0SF',
    'Column Piers':'0CP',
    'Columns Supporting Floors':'CSF',
    'Floor Girders and Beams':'FGB',
    'Floor Trusses':'0FT',
    'Floor Joists':'0FJ',
    'Columns Supporting Roofs':'CSR',
    'Roof Girders and Beams':'RGB',
    'Roof Trusses':'0RT',
    'Roof Joists':'0RJ',
    'Parking Bumpers':'0PB',
    'Precast Concrete Stair Treads':'PCS',
    'Roof Curbs':'0RC',
    'Exterior Wall Construction':'EWC',
    'Composite Decking':'CPD',
    'Cast-in-Place concrete':'CIC',
    'Floor Structural Frame':'FSF',
    'Associated Metal Fabrications':'AMF',
    'Floor Construction Supplementary Components':'FCS',
    'Roof Construction Supplementary Components':'RCS',
    'Residential Elevators':'0RE',
    'Vegetated Low-Slope Roofing':'VLR',
    'Swimming Pools':'SWP',
    'Excavation Soil Anchors':'ESA',
    'Floor Trusses':'FTS',
    'Roof Window and Skylight Performance':'RWS',
    'Rainwater Storage Tanks':'RST',
    'Gray Water Tanks':'GWT'}.items()
}

additional_categories_map['0FT'] = 'Floor Trusses'

headings = ['Building Identifier',
            'Country',
            'Gross Floor Area',
 'City',
 'Quality / Stage of Data',
 'Construction Date',
 'Building Type',
 'Contributor']

# Get Heeren & Fishman (2019) Material Names

Using the Mapping Names datasheet, find the relevant entry and identify the name.

In [8]:
def get_material_name(l):
    code = l.split('_')[2]
    if code.count('.') == 2:
        code = '.'.join(code.split('.')[1:])
    else:
        code = code.split('.')[0]
    return mapper[mapper.Code.str.contains(code.strip())].Category.values[0]

# Get external data from datasheets

The following functions obtain data from the external datasheets, if available.

In [9]:
def get_gdp(v):
    if v['Country'] == 'CA':
        idx = 33
    else:
        idx = 249
    if v['Construction Date'] > 2019:
        return gdp.at[idx,'2019']
    if v['Construction Date'] < 1960:
        return None
    return gdp.at[idx,str(int(v['Construction Date']))]

In [10]:
def get_urbanization(v):
    try:
        range_low = int(np.floor(v['Construction Date']/5)*5)
        range_high = int(range_low+5)
        return urbanization.at[v['Country'],f'{range_low}-{range_high}']
    except:
        return None

In [11]:
def get_distance_equator(v):
    if v['City'] == 'RIC':
        return 5466
    if v['City'] == 'TOR':
        return 4859
    if v['City'] == 'WIN':
        return 4703
    if v['City'] == 'NEW':
        return 4527
    return None

In [12]:
def get_climate(v):
    if v['City'] == 'RIC':
        return 'Cfb'
    if v['City'] == 'TOR':
        return 'Dfb'
    if v['City'] == 'WIN':
        return 'Dfa'
    if v['City'] == 'NEW':
        return 'Cfa'
    return None

# Number of floors
This helper function simply counts the number of floors in a given building.

In [13]:
def number_of_floors(v):
    notna = set()
    for ki,vi in v.items():
        if vi == vi and '_' in ki:
            notna.add(ki.split('_')[0])
    return len(notna)

# Main conversion loop
For each row, map the values in our format to the values in the Heeren & Fishman (2019) format.

If the conversion is 1:1, it is not explained below. Otherwise, a comment explains the motivation.

In [14]:
results = []
for k,v in tqdm(df.iterrows(),total=len(df)):
    output = {}
    output['sid'] = v['Building Identifier']
    output['Country'] = v['Country']
    output['Region'] = v['City']
    output['construction_period_start'] = v['Construction Date']
    output['construction_period_end'] = v['Construction Date']
        
    #Using our helper function get_material_name(), we parse the column and find the relevant mapping in the mapper datasheet.
    for l,w in v.items():
        if l not in headings and w==w:
            split = re.split('[_\.\ ]',l)
            out = get_material_name(l)
            if not out:
                continue
            else:
                if out.strip() in output.keys():
                    output[out.strip()] += w
                else:
                    output[out.strip()] = w
    
    output['building_description'] = building_name_mapper[(building_name_mapper == v['Building Type']).values]['Building Type'].values[0]
    output['no_floors'] = number_of_floors(v)
    output['occupation'] = building_name_mapper[(building_name_mapper == v['Building Type']).values]['Occupation'].values[0]
    output['building_type'] = v['Building Type']
    output['measurement_type'] = 'Case study'
    output['urban_rural'] = 'Urban'
    output['floor_area_type'] = v['Gross Floor Area']
    output['Authors'] = 'Shoshanna Saxe Research Group'
    output['publication_year'] = 2021
    output['publication_title'] = 'A Construction Classification System Database for Understanding Resource Use in Building Construction'
    output['publication_outlet'] = None #This will be updated when our submission is accepted.
    output['copyright_publication'] = None #This will be updated when our submission is accepted.
    output['DOI'] = None #This will be updated when our submission is accepted.
    output['URL'] = None #This will be updated when our submission is accepted.
    output['comment_scope'] = None #Not available in our database.
    output['comment_conversion'] = None #Not available in our database.
    output['comment_aggregation'] = 'NA' #Not applicable in our database.
    output['comment_primary'] = 'Material quantities were obtained through performing take offs directly from the building drawings.'
    output['comment_secondary'] = None #Not available in our database.
    output['data_entry'] = None #Not available in our database.
    output['global_region'] = 'Northern America' #All of our cities are in North America.
    output['climate_classification'] = get_climate(v)
    output['equator_distance'] = get_distance_equator(v)
    output['heating_degree_days_0'] = None #We were not able to obtain this data for a significant number of the years in our database.
    output['heating_degree_days_1'] = None #We were not able to obtain this data for a significant number of the years in our database.
    output['cooling_degree_days_0'] = None #We were not able to obtain this data for a significant number of the years in our database.
    output['cooling_degree_days_1'] = None #We were not able to obtain this data for a significant number of the years in our database.
    
    #As our entries are either in the US or Canada, we provide the values for those two countries explicitly.
    output['country_land_area'] = 9984670 if v['Country'] == 'CA' else 9833517
    
    #We obtain these values from our external datasets.
    output['population_0'] = population[str(int(v['Construction Date']))].values[0] if str(int(v['Construction Date'])) in population.columns else None
    output['population_1'] = output['population_0']
    output['urbanization_0'] = get_urbanization(v)
    output['urbanization_1'] = output['urbanization_0']
    output['gdp_0'] = get_gdp(v)
    output['gdp_1'] = output['gdp_0']
    
    #We were not able to obtain this data for a significant number of the years in our database.
    output['hdi_0'] = None
    output['hdi_1'] = None
    results.append(output)
    
#Convert to a Pandas dataframe.
results = pd.DataFrame(results)
results.set_index('sid',inplace=True)

100%|███████████████████████████████████████████| 72/72 [00:12<00:00,  5.90it/s]


In [15]:
df.to_csv('converted_dataset.csv')